<a href="https://colab.research.google.com/github/ayusingh-54/Multimodal-RAG-System/blob/main/multimodal_rag_with_gemini_langchain_and_google_ai_studio_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade  langchain langchain-google-genai "langchain[docarray]" faiss-cpu pypdf

In [ ]:
import os
import requests
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Markdown

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain.schema.document import Document
from langchain_community.document_loaders import TextLoader

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
def load_model(model_name):
  if model_name=="gemini-pro":
    llm = ChatGoogleGenerativeAI(model="gemini-live-2.5-flash-preview")
  else:
    llm=ChatGoogleGenerativeAI(model="gemini-live-2.5-flash-preview")

  return llm


In [ ]:
model_text=load_model("gemini-pro")

In [ ]:
model_text.invoke("capital of india").content

NotFound: 404 models/gemini-live-2.5-flash-preview is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
model_text(
    [
        HumanMessage(content="Answer with Simple 'Yes' or 'No'. Question: Is apple a Fruit?")
    ]
).content

In [ ]:
def get_image(url,filename,extension):
  content = requests.get(url).content
  with open(f'/content/{filename}.{extension}', 'wb') as f:
    f.write(content)
  image = Image.open(f"/content/{filename}.{extension}")
  image.show()
  return image



In [ ]:
image = get_image("https://static.nike.com/a/images/t_PDP_1728_v1/f_auto,q_auto:eco/1705ca64-fbc8-4b79-a451-4ab77760c219/dunk-low-older-shoes-C7T1cx.png",
                  "nike-shoes",
                  "png")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
vision_model=load_model("gemini-pro-vision")

In [ ]:
prompt="give me summary of this image in 5 words"

In [ ]:
message= HumanMessage(
    content=[
         {
            "type": "text",
            "text": prompt,
        },
        {

            "type": "image_url", "image_url": image
        }
    ]
)

In [ ]:
print(vision_model.invoke([message]).content)

In [ ]:
loader = TextLoader("/content/nike_shoes.txt")
print(loader.load()[0].page_content)

In [ ]:
text=loader.load()[0].page_content

In [ ]:
def get_text_chunks_langchain(text):
  text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=10)
  docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
  return docs

In [ ]:
docs = get_text_chunks_langchain(text)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vectorstore = FAISS.from_documents(docs,embedding=embeddings)

In [ ]:
retriever=vectorstore.as_retriever()

In [ ]:
retriever.invoke("Nike slide/sandal.")

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
llm_vision = load_model("gemini-pro-vision")

In [ ]:
llm_text = load_model("gemini-pro")

In [ ]:
template = """
```
{context}
```

{query}


Provide brief information and store location.
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm_text
    | StrOutputParser()
)

In [ ]:
result = rag_chain.invoke("can you give me a detail of nike sandal?")

In [ ]:
display(Markdown(result))

In [ ]:
rag_chain

In [ ]:
full_chain = (
    RunnablePassthrough() | llm_vision | StrOutputParser() | rag_chain
)

In [ ]:
full_chain

In [ ]:
url_1 = "https://static.nike.com/a/images/t_PDP_1728_v1/f_auto,q_auto:eco/252f2db6-d426-4931-80a0-8b7f8f875536/calm-slides-K7mr3W.png"

In [ ]:
image = get_image(url_1, "nike3", "png")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Provide information on given sandle image Brand and model.",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": image},
    ]
)


In [ ]:
result = full_chain.invoke([message])

In [ ]:
display(Markdown(result))